In [1]:
import helpers
import numpy as np
from scipy.stats import mode

# file_name = input("Enter the dataset file name: ")
file_name = "project3_dataset1.txt"

In [2]:
X, y, _ = helpers.import_txt(file_name)

X = np.array(X)
y = np.array(y).reshape((-1, 1))

In [3]:
def metric_computation(validation_labels,predicted_labels):
    tp,tn,fp,fn = 0.0,0.0,0.0,0.0
    for i in range(len(validation_labels)):
        if(validation_labels[i]==1.0 and predicted_labels[i]==1.0):
            tp += 1
        elif(validation_labels[i]==0.0 and predicted_labels[i]==1.0):
            fp += 1
        elif(validation_labels[i]==0.0 and predicted_labels[i]==0.0):
            tn += 1
        else:
            fn += 1
    
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fmeasure = (2*recall*precision)/(recall+precision)

    return accuracy,precision,recall,fmeasure

In [4]:
def cross_validation_split(k, X, y):
    
    permutation = np.random.permutation(X.shape[0])

    shuffled_X = X[permutation]
    shuffled_y = y[permutation]
    
    n = shuffled_X.shape[0] // k

    folds = []

    start = 0

    for i in range(k):
        fold = [shuffled_X[start:start+n], shuffled_y[start:start+n]]
        folds.append(fold)
        start = start+n

    rem = shuffled_X.shape[0] % k

    if rem > 0:
        last_X = folds[-1][0]
        last_y = folds[-1][1]

        folds[-1][0] = np.concatenate((last_X, shuffled_X[-1:-rem-1:-1]))
        folds[-1][1] = np.concatenate((last_y, shuffled_y[-1:-rem-1:-1]))

    # for 10 times do cross validation excluding ith fold
    print([fold[0].shape for fold in folds])
    
    return folds
    

In [5]:
def gini(rows):
    
    unique, counts = np.unique(np.array(rows)[:, -1], return_counts=True)
    counts = dict(zip(unique, counts))
    
    impurity = 1
    for label in counts:
        impurity -= (counts[label] / float(len(rows)))**2
    return impurity

In [6]:
def info_gain(left, right, parent_gini):
    p = float(len(left)) / (len(left) + len(right))
    return parent_gini - p * gini(left) - (1 - p) * gini(right)

In [7]:
# handle both numeric and categorical data
class Question:

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, row):
        
        val = row[self.column]
        
        if isinstance(val, float):
            return val >= self.value
        
        elif isinstance(val, str) or self.column == len(row)-1:
            return val == self.value
    
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        
        if isinstance(self.value, float):
            condition = ">="
        
        return "Is %s %s %s?" % (self.column, condition, str(self.value))

In [8]:
class DecisionTreeClassifier:
    
    def __init__(self, n_features=None):
        self.n_features = n_features
        self.tree = None
    
    def fit(self, X, y):
        
        self.data = np.concatenate((X, y), axis=1)
        self.tree = self._make_tree(self.data)
    
    def predict(self, X):
        
        predictions = np.empty((X.shape[0], 1))
        
        for idx, row in enumerate(X):
            
            label = self._classify(row, self.tree)
            
            predictions[idx] = label
        
        return predictions.flatten()
    
    def _make_tree(self, subset, level=0):
        
        best_gain, best_question = self._best_split(subset)
        
        
        if best_gain == 0:
            return self._make_leaf_node(subset)

        true_set, false_set = self._partition(subset, best_question)

        true_branch = self._make_tree(true_set, level+1)

        false_branch = self._make_tree(false_set, level+1)

        return self._make_decision_node(best_question, true_branch, false_branch)
    
    def _best_split(self, subset):
        
        best_gain = 0
        best_question = None
        selected_cols = set()
        
        curr_gini = gini(subset)
        num_cols = len(subset[0]) - 1
        
        columns = list(range(num_cols))
        
        uniq_cols = set()
        
        if self.n_features:
            while len(uniq_cols) < self.n_features:
                sample_col = np.random.choice(num_cols, 1)[0]
                if  sample_col not in uniq_cols:
                    uniq_cols.add(sample_col)

            columns = list(uniq_cols)

        for col in columns:
            
            if col not in selected_cols:
                values = set([row[col] for row in subset])

                for val in values:

                    curr_question = Question(col, val)

                    true_subset, false_subset = self._partition(subset, curr_question)

                    if len(true_subset) == 0 or len(false_subset) == 0:
                        continue

                    curr_gain = info_gain(true_subset, false_subset, curr_gini)

                    if curr_gain >= best_gain:
                        best_gain, best_question = curr_gain, curr_question
                        selected_cols.add(col)

        return best_gain, best_question
    
    
    
    def _partition(self, subset, question):
        
        true_subset, false_subset = [], []
        for row in subset:
            if question.match(row):
                true_subset.append(row)
            else:
                false_subset.append(row)
        return true_subset, false_subset
    
    def _classify(self, row, node):

        if node["type"] == "leaf":
            return node["prediction"]
        else:
            if node["question"].match(row):
                return self._classify(row, node["true_branch"])
            else:
                return self._classify(row, node["false_branch"])
    
    def _make_leaf_node(self, subset):
    
        unique, counts = np.unique(np.array(subset)[:, -1], return_counts=True)

        m = counts.argmax()

        return {
            "type": "leaf",
            "prediction": float(unique[m])
        }

    def _make_decision_node(self, question, true_branch, false_branch):

        return {
            "type": "decision",
            "question": question,
            "true_branch": true_branch,
            "false_branch": false_branch
        }
        
    def _print_tree(self):
        
        tree_repr = []
        
        self._print_tree_recursive(self.tree, tree_repr)
    
        return "\n".join(tree_repr)
    
    def _print_tree_recursive(self, node, tree_repr, spacing=""):
    
        # Base case: we've reached a leaf
        if node["type"] == "leaf":
            tree_repr.append(spacing + "Predict" + str(node["prediction"]))
            return

        # Print the question at this node
        tree_repr.append(spacing + str(node["question"]))

        # Call this function recursively on the true branch
        tree_repr.append(spacing + '--> True:')
        self._print_tree_recursive(node["true_branch"], tree_repr, spacing + "  ")

        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        self._print_tree_recursive(node["false_branch"], tree_repr, spacing + "  ")
        
        
    

In [9]:
class RandomForestClassifier:
    
    def __init__(self, n_estimators=1, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = []
    
    def fit(self, X, y):
        
        
        for _ in range(self.n_estimators):
            
            # TODO: don't hardcode bootstrap size
            data = np.concatenate((X, y), axis=1)
            data_bootstrapped = self._get_bootstrap_set(data, data.shape[0])
            
            X_bootstrapped, y_bootstrapped = data_bootstrapped[:, :-1], data_bootstrapped[:, -1].reshape((-1, 1))
            
            print(X_bootstrapped.shape, y_bootstrapped.shape)
            
            # TODO: add max_depth
            tree = DecisionTreeClassifier()
            
            # TODO: pretty print tree
            tree.fit(X_bootstrapped, y_bootstrapped)
            
            self.trees.append(tree)
            
            
            
    
    def predict(self, X):
        predictions = []
        
        for tree in self.trees:
            pred = tree.predict(X).reshape((-1, 1))
            
            predictions.append(pred)
        print(np.array(predictions).shape)
        
        # TODO: do mode calculation from scratch
        return mode(np.array(predictions), axis=0)[0]

            
    def _get_bootstrap_set(self, data, dataset_length):
    
        indices = np.random.randint(0, data.shape[0], dataset_length)

        subset = data[indices].copy()

        print(subset[:5])
        
        return subset


# get_bootstrap_set(X, X.shape[0] // 10)

# pick random set of features
# make n number of decision trees
# do majority voting

In [10]:
# clf = RandomForestClassifier(n_estimators=2, n_features=20)
# clf.fit(X, y)
# predictions = clf.predict(X)

In [11]:
folds = cross_validation_split(10, X, y)

[(56, 30), (56, 30), (56, 30), (56, 30), (56, 30), (56, 30), (56, 30), (56, 30), (56, 30), (65, 30)]


In [12]:
for i in range(10):
    X_train = np.array([]).reshape(0, X.shape[1])
    y_train = np.array([]).reshape(0, 1)

    # train on remaining 9 folds
    for idx, fold in enumerate(folds):
        if idx != i:
            [x_fold, y_fold] = fold
            X_train = np.vstack((X_train, x_fold))
            y_train = np.vstack((y_train, y_fold))
    # predict on one fold
    [X_test, y_test] = folds[i]
    
        
    classifier = DecisionTreeClassifier(n_features=10)

    classifier.fit(X_train, y_train)
    
    print(classifier._print_tree())

    predictions = classifier.predict(X_test)
    
    accuracy, precision, recall, fmeasure = metric_computation(y_test, predictions)
    
    print(accuracy, precision, recall, fmeasure)
    
    

      --> False:
        --> False:
    --> False:
      --> False:
        --> False:
  --> False:
    --> False:
--> False:
        --> False:
          --> False:
      --> False:
        --> False:
    --> False:
      --> False:
  --> False:
        --> False:
          --> False:
      --> False:
          --> False:
        --> False:
            --> False:
          --> False:
              --> False:
            --> False:
    --> False:
Is 27 >= 0.1445?
--> True:
  Is 23 >= 697.7?
  --> True:
    Is 27 >= 0.151?
    --> True:
      Is 7 >= 0.05602000000000001?
      --> True:
        Predict1.0
        Is 20 >= 16.23?
        --> True:
          Predict1.0
          Predict0.0
      Is 27 >= 0.1505?
      --> True:
        Predict0.0
        Is 13 >= 24.91?
        --> True:
          Predict1.0
          Predict0.0
    Is 28 >= 0.4154?
    --> True:
      Predict1.0
      Predict0.0
  Is 20 >= 16.84?
  --> True:
    Is 11 >= 0.8554?
    --> True:
      Is 24 >= 0.1021?
     

      --> False:
        --> False:
    --> False:
  --> False:
      --> False:
    --> False:
            --> False:
          --> False:
        --> False:
      --> False:
            --> False:
          --> False:
        --> False:
          --> False:
--> False:
    --> False:
  --> False:
          --> False:
        --> False:
      --> False:
    --> False:
          --> False:
            --> False:
        --> False:
      --> False:
Is 22 >= 106.0?
--> True:
  Is 20 >= 17.52?
  --> True:
    Is 24 >= 0.08822?
    --> True:
      Is 26 >= 0.1932?
      --> True:
        Predict1.0
        Is 29 >= 0.06515?
        --> True:
          Predict1.0
          Predict0.0
      Predict0.0
    Is 27 >= 0.1521?
    --> True:
      Is 21 >= 22.4?
      --> True:
        Predict1.0
        Predict0.0
      Is 21 >= 27.66?
      --> True:
        Is 24 >= 0.1131?
        --> True:
          Is 29 >= 0.1023?
          --> True:
            Is 29 >= 0.1027?
            --> True:
       

    --> False:
      --> False:
  --> False:
        --> False:
      --> False:
    --> False:
--> False:
    --> False:
      --> False:
  --> False:
        --> False:
            --> False:
          --> False:
      --> False:
          --> False:
        --> False:
          --> False:
    --> False:
          --> False:
        --> False:
            --> False:
          --> False:
              --> False:
            --> False:
      --> False:
Is 23 >= 888.3?
--> True:
  Is 7 >= 0.051820000000000005?
  --> True:
    Is 16 >= 0.01715?
    --> True:
      Predict1.0
      Is 19 >= 0.00233?
      --> True:
        Predict0.0
        Predict1.0
    Is 21 >= 20.24?
    --> True:
      Is 15 >= 0.01805?
      --> True:
        Is 20 >= 18.13?
        --> True:
          Predict0.0
          Predict1.0
        Predict1.0
      Predict0.0
  Is 27 >= 0.1565?
  --> True:
    Is 13 >= 22.69?
    --> True:
      Predict1.0
      Is 1 >= 18.83?
      --> True:
        Predict1.0
        Pr